## Workshop "Data Ingestion with dlt": Homework

### Dataset & API
We’ll use NYC Taxi data via the same custom API from the workshop:

- Base API URL:

https://us-central1-dlthub-analytics.cloudfunctions.net/data_engineering_zoomcamp_api

- Data format: Paginated JSON (1,000 records per page).
- API Pagination: Stop when an empty page is returned.

## Question 1: dlt Version

In [3]:
#!pip install dlt[duckdb]
import dlt 

print("dlt verison: ", dlt.__version__)

dlt verison:  1.7.0


## Question 2: Define & Run the Pipeline (NYC Taxi API)
    
Use dlt to extract all pages of data from the API.
    
Steps:

1. Use the @dlt.resource decorator to define the API source.
2. Implement automatic pagination using dlt's built-in REST client.
3. Load the extracted data into DuckDB for querying.

In [12]:
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import PageNumberPaginator

@dlt.resource(name='rides')
def ny_taxi():
    client = RESTClient(
        base_url="https://us-central1-dlthub-analytics.cloudfunctions.net",
        paginator=PageNumberPaginator(base_page=1, total_path=None)
    )

    for page in client.paginate("data_engineering_zoomcamp_api"):
        yield page

pipeline = dlt.pipeline(pipeline_name = "ny_taxi_pipeline",
                        destination="duckdb",
                        dataset_name="ny_taxi_data")

In [13]:
load_info = pipeline.run(ny_taxi, write_disposition="replace")
print(load_info)

Pipeline ny_taxi_pipeline load step completed in 2.09 seconds
1 load package(s) were loaded to destination duckdb and into dataset ny_taxi_data
The duckdb destination used duckdb:///c:\Users\swang\projects\data_engineer_zoomcamp\week_4\dbt workshop\ny_taxi_pipeline.duckdb location to store data
Load package 1740094897.1341891 is LOADED and contains no failed jobs


Start a connection to your database using native duckdb connection and look what tables were generated:"""

In [14]:
import duckdb
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

# A database '<pipeline_name>.duckdb' was created in working directory so just connect to it

# Connect to the DuckDB database
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# Set search path to the dataset
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# Describe the dataset
conn.sql("DESCRIBE").df()


,database,schema,name,column_names,column_types,temporary
0,ny_taxi_pipeline,ny_taxi_data,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,ny_taxi_pipeline,ny_taxi_data,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,ny_taxi_pipeline,ny_taxi_data,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,ny_taxi_pipeline,ny_taxi_data,rides,"[end_lat, end_lon, fare_amt, passenger_count, ...","[DOUBLE, DOUBLE, DOUBLE, BIGINT, VARCHAR, DOUB...",False


## Question 3: Explore the loaded data

Inspect the table ride:

In [17]:
df = pipeline.dataset(dataset_type="default").rides.df()
df.shape

(10000, 18)

## Question 4: Trip Duration Analysis

Run the SQL query below to:

- Calculate the average trip duration in minutes.

In [18]:
with pipeline.sql_client() as client:
    res = client.execute_sql(
            """
            SELECT
            AVG(date_diff('minute', trip_pickup_date_time, trip_dropoff_date_time))
            FROM rides;
            """
        )
    # Prints column values of the first row
    print(res)

[(12.3049,)]
